In [ ]:
import numpy as np
import random

from FPU import cmul, cadd
from utils import conj, read_binary, binary_to_fp16, store_binary, generate_fp16, fp16_to_binary, int_to_binary, binary_to_int
from ed import ed
from hilbert import hilbert_manual_conv

# Basic functions

In [ ]:
def delayseq(data, lag: int, mode: str = "zero"):
    """
    Apply delay (positive lag) or advance (negative lag) to a sequence.

    Supports both:
    - Real sequences (list of np.float16)
    - Complex sequences (list of [real, imag] pairs)

    Output length is always equal to the input length.

    Args:
        data: Input sequence (real or complex).
        lag:  Number of samples to delay (positive) or advance (negative).
        mode:
            "zero" → fill shifted positions with zeros (default).
            "wrap" → perform circular (cyclic) shift.

    Returns:
        Delayed (or advanced) sequence of the same length.
    """
    if not data:
        return []

    # Detect if the input is complex
    is_complex = isinstance(data[0], (list, tuple)) and len(data[0]) == 2
    n = len(data)

    # Circular shift mode
    if mode == "wrap":
        k = lag % n
        return data[-k:] + data[:-k] if k != 0 else data.copy()

    if mode != "zero":
        raise ValueError('mode must be "zero" or "wrap"')

    # Zero-fill mode
    zero_val = [np.float16(0.0), np.float16(0.0)] if is_complex else np.float16(0.0)

    # Positive lag → delay to the right (zeros at the front)
    if lag > 0:
        fill = [zero_val] * min(lag, n)
        tail = data[: max(0, n - lag)]
        return fill + tail

    # No lag
    if lag == 0:
        return data.copy()

    # Negative lag → advance to the left (zeros at the end)
    lag_abs = min(-lag, n)
    head = data[lag_abs:]
    fill = [zero_val] * lag_abs
    return head + fill

# Custom Kernel

In [60]:
# Square, Mean, Conj, Conv, Abs
def square_complex(data_in):
    data_out = []
    for idx, val in enumerate(data_in):
        real = val[0]
        imag = val[1]

        real_out = real*real - imag*imag
        imag_out = 2*real*imag
        data_out.append([real_out, imag_out])
    return data_out

def mean_complex(data_in):
    real_out = np.float16(0)
    imag_out = np.float16(0)
    for idx, val in enumerate(data_in):
        real = val[0]
        imag = val[1]

        real_out += real
        imag_out += imag
    data_out = [real_out/len(data_in), imag_out/len(data_in)]
    return data_out

def conj(data_in):
    data_out = []
    for idx, val in enumerate(data_in):
        real = val[0]
        imag = val[1]

        real_out = real
        imag_out = -imag
        data_out.append([real_out, imag_out])
    return data_out

def abs_complex(data_in):
    data_out = []
    for idx, val in enumerate(data_in):
        real = val[0]
        imag = val[1]

        real_out = np.float16(real*real + imag*imag)
        imag_out = np.float16(0)
        data_out.append([real_out, imag_out])
    return data_out

In [63]:
sample = generate_fp16(2, complex=True)
sample

[[np.float16(0.9185), np.float16(-0.925)],
 [np.float16(-0.1749), np.float16(-0.3303)]]

In [62]:
abs_complex(sample)

[[np.float16(1.699), np.float16(0.0)], [np.float16(0.1398), np.float16(0.0)]]

In [39]:
square_out = square(sample)

## CP Detect

## IA

## QAM Estimation
- Equalization